# Chroma

Chroma is a AI-native open source vector database focused on developer productivity and Happiness.Chroma is licensed under Apache 2.0

In [3]:
# build 
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [4]:
loader = TextLoader('../3.2_DataIngestion/speech.txt')
data = loader.load()
data

[Document(metadata={'source': '../3.2_DataIngestion/speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of 

In [5]:
#Split

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
splits = text_splitter.split_documents(data)
splits

[Document(metadata={'source': '../3.2_DataIngestion/speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.'),
 Document(metadata={'source': '../3.2_DataIngestion/speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'),
 Document(

In [6]:
embeddings = OllamaEmbeddings(model='gemma:2b')
vectordb = Chroma.from_documents(documents=splits,embedding=embeddings)

C:\Users\chara\AppData\Local\Temp\ipykernel_23832\2072878273.py:1: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model='gemma:2b')


In [8]:
## query it

query = "What does the speaker believe is the main reason the united states should enter the war?"
docs = vectordb.similarity_search(query)
docs[0].page_content

'for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

In [10]:
## Saving to the disk

vectordb = Chroma.from_documents(documents=splits, embedding=embeddings,persist_directory='./chroma_db')

In [11]:
# Load from disk

db2 = Chroma(persist_directory='./chroma_db',embedding_function=embeddings)

In [12]:
docs = db2.similarity_search(query)
docs[0].page_content

'for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

In [13]:
## Similarity Search With Score

docs = vectordb.similarity_search_with_score(query)
docs

[(Document(id='73af081d-13bc-4954-bfe3-70ee404b8a8b', metadata={'source': '../3.2_DataIngestion/speech.txt'}, page_content='for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
  3175.79150390625),
 (Document(id='9db77f5d-7ab3-4f82-9623-9ae26f8106be', metadata={'source': '../3.2_DataIngestion/speech.txt'}, page_content='government in the hour of test. They are, most of them, as true and loyal Americans as if they had never known any other fealty or allegiance. They will be prompt to stand with us in rebuking and restraining the few who may be of a different mind and purpose. If there should be disloyalty, it will be dealt with with a firm hand of stern repression; but, if it lifts its head at all, it will lift it only here and there and without coun

In [14]:
## Retriever Option

retriever = vectordb.as_retriever()
retriever.invoke(query)[0].page_content


'for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'